In [35]:
#initial imports
import pandas as pd

LTPATH = r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\bonds\concatenated_1823.xlsx"
DTPATH = r""


In [36]:
#clean demographic data
# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\demog\cc-est2022-all.csv'

# Read the Excel file into a pandas DataFrame
demog = pd.read_csv(excel_file, encoding='latin1')

# Display the columns
print("Columns in the Excel file:")
for column in demog.columns:
    print(column)

Columns in the Excel file:
SUMLEV
STATE
COUNTY
STNAME
CTYNAME
YEAR
AGEGRP
TOT_POP
TOT_MALE
TOT_FEMALE
WA_MALE
WA_FEMALE
BA_MALE
BA_FEMALE
IA_MALE
IA_FEMALE
AA_MALE
AA_FEMALE
NA_MALE
NA_FEMALE
TOM_MALE
TOM_FEMALE
WAC_MALE
WAC_FEMALE
BAC_MALE
BAC_FEMALE
IAC_MALE
IAC_FEMALE
AAC_MALE
AAC_FEMALE
NAC_MALE
NAC_FEMALE
NH_MALE
NH_FEMALE
NHWA_MALE
NHWA_FEMALE
NHBA_MALE
NHBA_FEMALE
NHIA_MALE
NHIA_FEMALE
NHAA_MALE
NHAA_FEMALE
NHNA_MALE
NHNA_FEMALE
NHTOM_MALE
NHTOM_FEMALE
NHWAC_MALE
NHWAC_FEMALE
NHBAC_MALE
NHBAC_FEMALE
NHIAC_MALE
NHIAC_FEMALE
NHAAC_MALE
NHAAC_FEMALE
NHNAC_MALE
NHNAC_FEMALE
H_MALE
H_FEMALE
HWA_MALE
HWA_FEMALE
HBA_MALE
HBA_FEMALE
HIA_MALE
HIA_FEMALE
HAA_MALE
HAA_FEMALE
HNA_MALE
HNA_FEMALE
HTOM_MALE
HTOM_FEMALE
HWAC_MALE
HWAC_FEMALE
HBAC_MALE
HBAC_FEMALE
HIAC_MALE
HIAC_FEMALE
HAAC_MALE
HAAC_FEMALE
HNAC_MALE
HNAC_FEMALE


In [37]:
demog = demog[demog['AGEGRP'] == 0]
demog = demog[demog['YEAR'] == 1]

demog['CTYNAME'] = demog['CTYNAME'].apply(lambda x: x[:-7] if x.endswith(" County") else x)

demog['PropWhiteTC'] = (demog['WAC_MALE'] + demog['WAC_FEMALE']) / demog['TOT_POP']
demog['PropBlackTC'] = (demog['BAC_MALE'] + demog['BAC_FEMALE']) / demog['TOT_POP']
demog['PropAsianTC'] = (demog['AAC_MALE'] + demog['AAC_FEMALE']) / demog['TOT_POP']
demog['PropHispaTC'] = (demog['H_MALE'] + demog['H_FEMALE']) / demog['TOT_POP']
demog['PropWhiteTA'] = (demog['WA_MALE'] + demog['WA_FEMALE']) / demog['TOT_POP']
demog['PropBlackTA'] = (demog['BA_MALE'] + demog['BA_FEMALE']) / demog['TOT_POP']
demog['PropAsianTA'] = (demog['AA_MALE'] + demog['AA_FEMALE']) / demog['TOT_POP']

demog = demog.loc[:, ['CTYNAME', 'STNAME', 'STATE', 'COUNTY','PropWhiteTC',"TOT_POP",
                      'PropBlackTC', 'PropAsianTC', 'PropHispaTC',
                      'PropWhiteTA', 'PropAsianTA', 'PropBlackTA']]

demog.head()


,CTYNAME,STNAME,STATE,COUNTY,PropWhiteTC,TOT_POP,PropBlackTC,PropAsianTC,PropHispaTC,PropWhiteTA,PropAsianTA,PropBlackTA
0,Autauga,Alabama,1,1,0.776742,58802,0.214500,0.016921,0.031342,0.759107,0.011751,0.204466
76,Baldwin,Alabama,1,3,0.891582,231761,0.096440,0.015943,0.047105,0.872705,0.011063,0.087405
152,Barbour,Alabama,1,5,0.501943,25224,0.491635,0.007057,0.047613,0.489613,0.004599,0.481367
228,Bibb,Alabama,1,7,0.779776,22300,0.217848,0.004709,0.028834,0.767444,0.002601,0.210404
304,Blount,Alabama,1,9,0.970455,59130,0.022577,0.006021,0.095890,0.957128,0.003754,0.017216


In [38]:
#merge and clean bond data

# Replace 'your_excel_file.xlsx' with the path to your Excel file

excel_file = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\bonds\combined_file.xlsx'
# Read the Excel file into a pandas DataFrame
muni = pd.read_excel(excel_file)

# Display the columns
print("Columns in the Excel file:")
for column in muni.columns:
    print(column)

Columns in the Excel file:
8-digit CUSIP
9-digit CUSIP
State of Issuer
County
Joint action
Name of Issuer
Gross Spread (per thousand) (USD)
Reported Gross Spread (per thousand) (USD)
SDC Estimated Gross Spread (per thousand) (USD)
Management fee (USD)
Underwriters' fee (USD)
Bid Type
Security
Sinking fund flag
Callable Issue
All Managers Code
All Use of Proceeds
All Use of Proceeds - Bond Buyer
Asset-Backed Indicator Flag (Y/N)
Auction Rate Flag
Bank Eligible
Bank Qualified
Book entry
Charter School Flag (Y/N)
Climate Bond (Y/N)
Composite Amount (USD Millions)
Credit Enhancer Type
Deals with Financial Advisor (Y/N)
Deals with Underwriter Counsel (Y/N)
Fitch's Enhanced Long Term Rating
Fitch's Enhanced Short Term Rating
Fitch's Insured Long Term Rating
Fitch's Insured Short Term Rating
Fitch's Underlying Long Term Rating
Fitch's Underlying Short Term Rating
Insured Amount - Tranche (USD Millions)
Issuer Type
Moody's Enhanced Long Term Rating
Moody's Enhanced Short Term Rating
Moody's In

In [39]:
muni = muni.rename(columns={"State of Issuer": "STNAME"})

# Create a new DataFrame to hold the expanded rows
expanded_rows = []

# Iterate through each row in muni DataFrame
for index, row in muni.iterrows():
    # Initialize the status variable
    status = 0  # 0 means no change
    
    # Check if 'County' is a string and contains a slash
    if isinstance(row['County'], str) and '/' in row['County']:
        # Split the county names separated by a slash
        counties = row['County'].split('/')
        # Create a new row for each county
        for county in counties:
            new_row = row.copy()  # Create a copy of the original row
            new_row['County'] = county.strip()  # Update the 'County' column with the individual county
            expanded_rows.append(new_row)  # Append the new row to the list of expanded rows
        status = 1  # 1 means split
    elif isinstance(row['County'], float):
        status = 2  # 2 means it's a float
    
    # Add the status to the row
    row['Status'] = status
    
    # Append the row to the list of expanded rows
    expanded_rows.append(row)

# Create a new DataFrame from the list of expanded rows
expanded_muni = pd.DataFrame(expanded_rows)

muni = expanded_muni.rename(columns={"County": "CTYNAME"})

In [40]:
muni.head()

muni.rename(columns={"Fitch's Insured Long Term Rating": 'Fitch_Long', "Fitch's Insured Short Term Rating": 'Fitch_Short', "Moody's Insured Long Term Rating": 'Moody_Long', "Moody's Insured Short Term Rating": 'Moody_Short'}, inplace=True)

# Define a function to extract the highest Moody's rating from a string
def extract_highest_rating(s):
    ratings_order = ['Aaa', 'Aa1', 'Aa2', 'Aa3', 'A1', 'A2', 'A3', 
                     'Baa1', 'Baa2', 'Baa3', 'Ba1', 'Ba2', 'Ba3',
                     'Caa1', 'Caa2', 'Caa3', 'Ca', 'C','NR']
    max_rating = 'C'  # Default lowest rating
    for rating in ratings_order:
        if rating in s:
            max_rating = rating
            break  # Stop when the highest rating is found
    return max_rating


# Define a function to assign an ordinal number to each rating
def rating_to_number(rating):
    rating_to_num = {'Aaa': 1, 'Aa1': 2, 'Aa2': 3, 'Aa3': 4, 'A1': 5, 'A2': 6, 'A3': 7,
                     'Baa1': 8, 'Baa2': 9, 'Baa3': 10, 'Ba1': 11, 'Ba2': 12, 'Ba3': 13,
                     'Caa1': 14, 'Caa2': 15, 'Caa3': 16, 'Ca': 17, 'C': 18}
    return rating_to_num.get(rating, 18)  # Default value is 18 if rating not found

# Apply the functions to each row and create new columns 'Moody_High' and 'Moody_Number'
muni['Moody_High'] = muni['Moody_Long'].fillna('').apply(extract_highest_rating)
muni['Moody_Number'] = muni['Moody_High'].apply(rating_to_number)




print("Columns in the Excel file:")
for column in muni.columns:
    print(column)



Columns in the Excel file:
8-digit CUSIP
9-digit CUSIP
STNAME
CTYNAME
Joint action
Name of Issuer
Gross Spread (per thousand) (USD)
Reported Gross Spread (per thousand) (USD)
SDC Estimated Gross Spread (per thousand) (USD)
Management fee (USD)
Underwriters' fee (USD)
Bid Type
Security
Sinking fund flag
Callable Issue
All Managers Code
All Use of Proceeds
All Use of Proceeds - Bond Buyer
Asset-Backed Indicator Flag (Y/N)
Auction Rate Flag
Bank Eligible
Bank Qualified
Book entry
Charter School Flag (Y/N)
Climate Bond (Y/N)
Composite Amount (USD Millions)
Credit Enhancer Type
Deals with Financial Advisor (Y/N)
Deals with Underwriter Counsel (Y/N)
Fitch's Enhanced Long Term Rating
Fitch's Enhanced Short Term Rating
Fitch_Long
Fitch_Short
Fitch's Underlying Long Term Rating
Fitch's Underlying Short Term Rating
Insured Amount - Tranche (USD Millions)
Issuer Type
Moody's Enhanced Long Term Rating
Moody's Enhanced Short Term Rating
Moody_Long
Moody_Short
Moody's Underlying Long Term Rating
Moo

In [41]:
merged_df = pd.merge(muni, demog, on=['STNAME', 'CTYNAME'], how='left')

In [42]:
merged_df.head()
print(merged_df.dtypes)
typechangedf = merged_df.copy()
typechangedf['8-digit CUSIP'] = typechangedf['8-digit CUSIP'].astype(str)
#typechangedf=merged_df.astype(str)

8-digit CUSIP     object
9-digit CUSIP     object
STNAME            object
CTYNAME           object
Joint action        bool
                  ...   
PropAsianTC      float64
PropHispaTC      float64
PropWhiteTA      float64
PropAsianTA      float64
PropBlackTA      float64
Length: 76, dtype: object


In [43]:

#merged_df.to_stata(r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\stata\MuniDemo.dta",version=117)

In [44]:
race_and_tax =  pd.read_csv(r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\race_and_Taxes\race_and_tax.csv")


In [45]:

race_and_tax.rename(columns={'state': 'STNAME'}, inplace=True)
print(race_and_tax.head())

        STNAME  AntiAffirm  Racial Resent  Google Search  Change in Vote  \
0    Louisiana           1              1            2.0             6.0   
1  Mississippi           2              3            4.0             2.0   
2      Alabama           4              2            8.5             1.0   
3      Georgia           3              4           16.5             3.0   
4     Arkansas           6              7           14.5            13.5   

   Tweets   sum  number of bond issues  number by hbcu  tax privilege  
0     1.0  11.0                     57               4            6.0  
1     4.0  15.0                     46               9            5.0  
2     2.0  17.5                     89              14            5.0  
3    11.5  38.0                     99              14            6.0  
4     3.0  44.0                    107               3            7.0  


In [46]:
merged_df = pd.merge(typechangedf, race_and_tax, on=['STNAME'], how='left')

In [47]:
# Read the CSV file
file_path = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\demog\educationrates.csv'
df = pd.read_csv(file_path)

# Remove columns that include "Margin of Error" in their header
columns_to_remove = [col for col in df.columns if 'Margin of Error' in col or 'RACE' in col or 'Female' in col or 'Male' in col or 'POVERTY' in col or 'MEDIAN' in col or 'Total!' in col]
educdf_cleaned = df.drop(columns=columns_to_remove)

columns_with_only_X = [col for col in educdf_cleaned.columns if (educdf_cleaned[col] == '(X)').all()]
educdf_cleaned = educdf_cleaned.drop(columns=columns_with_only_X)

# Save the cleaned dataframe to a new CSV file
output_path = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\demog\cleaned_educrates.csv'



# Dictionary mapping original column names to human-readable names
rename_dict = {
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate": "18-24 Less than High School",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)": "18-24 High School Graduate",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree": "18-24 Some College or Associate's",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher": "18-24 Bachelor's or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade": "25+ Less than 9th Grade",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma": "25+ 9th to 12th No Diploma",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)": "25+ High School Graduate",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree": "25+ Some College No Degree",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree": "25+ Associate's Degree",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree": "25+ Bachelor's Degree",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree": "25+ Graduate or Professional Degree",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher": "25+ High School or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher": "25+ Bachelor's or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher": "25-34 High School or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher": "25-34 Bachelor's or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!High school graduate or higher": "35-44 High School or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!Bachelor's degree or higher": "35-44 Bachelor's or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!High school graduate or higher": "45-64 High School or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!Bachelor's degree or higher": "45-64 Bachelor's or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!High school graduate or higher": "65+ High School or Higher",
    "Estimate!!Percent!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!Bachelor's degree or higher": "65+ Bachelor's or Higher"
}

# Rename the columns
educdf_cleaned = educdf_cleaned.rename(columns=rename_dict)


# Split the 'Geographic Area Name' column into 'County Name' and 'State Name' columns
educdf_cleaned[['CTYNAME', 'STNAME']] = df['Geographic Area Name'].str.split(' County, ', expand=True)

# Drop the original 'Geographic Area Name' column if needed
educdf_cleaned = educdf_cleaned.drop(columns=['Geographic Area Name'])
educdf_cleaned.to_csv(output_path, index=False)
print(f"Columns removed: {columns_to_remove}")
print(f"Cleaned CSV file saved to: {output_path}")

Columns removed: ['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years', 'Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years', 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate', 'Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate', 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)', 'Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)', "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree", "Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree", "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher", "Margi

C:\Users\rra3\AppData\Local\Temp\ipykernel_34756\1049121674.py:3: DtypeWarning: Columns (118,119,122,123,124,125,132,133,134,135,136,137,138,139,168,169,170,171,192,193,194,195,222,223,224,225,228,229,230,231,234,235,236,237,242,243,244,245,374,375,378,379,380,381,388,389,390,391,392,393,394,395,424,425,426,427,448,449,450,451,484,485,486,487,490,491,492,493,494,495,496,497,498,499,500,501,630,631,644,645,646,647,648,649,650,651,680,681,682,683,704,705,706,707,740,741,742,743,746,747,748,749,750,751,752,753,754,755,756,757) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [48]:
# Read the CSV file
file_path = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\demog\poverty rates.csv'
df = pd.read_csv(file_path)

# Remove columns that include "Margin of Error" in their header
columns_to_remove = [col for col in df.columns if 'Margin of Error' in col or 'RACE' in col or 'Female' in col or 'Male' in col or 'MEDIAN' in col or 'years' in col or "UNRELATED" in col]
povdf_cleaned = df.drop(columns=columns_to_remove)

columns_with_only_X = [col for col in povdf_cleaned.columns if (povdf_cleaned[col] == '(X)').all()]
povdf_cleaned = povdf_cleaned.drop(columns=columns_with_only_X)

# Save the cleaned dataframe to a new CSV file
output_path = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\demog\cleaned_povrates.csv'

# Mapping of original column names to new human-readable names
new_column_names = {
    'Geography': 'Geography',
    'Geographic Area Name': 'Geographic Area Name',
    'Estimate!!Total!!Population for whom poverty status is determined': 'Total Population',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!50 percent of poverty level': 'Below 50% Poverty Level',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!125 percent of poverty level': 'Below 125% Poverty Level',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!150 percent of poverty level': 'Below 150% Poverty Level',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!185 percent of poverty level': 'Below 185% Poverty Level',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!200 percent of poverty level': 'Below 200% Poverty Level',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!300 percent of poverty level': 'Below 300% Poverty Level',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!400 percent of poverty level': 'Below 400% Poverty Level',
    'Estimate!!Total!!Population for whom poverty status is determined!!ALL INDIVIDUALS WITH INCOME BELOW THE FOLLOWING POVERTY RATIOS!!500 percent of poverty level': 'Below 500% Poverty Level',
    'Estimate!!Below poverty level!!Population for whom poverty status is determined': 'Below Poverty Level Population',
    'Estimate!!Percent below poverty level!!Population for whom poverty status is determined': 'Percent Below Poverty Level Population'
}

# Rename the columns
povdf_cleaned.rename(columns=new_column_names, inplace=True)

povdf_cleaned[['CTYNAME', 'STNAME']] = df['Geographic Area Name'].str.split(' County, ', expand=True)

# Drop the original 'Geographic Area Name' column if needed
povdf_cleaned = povdf_cleaned.drop(columns=['Geographic Area Name'])
povdf_cleaned.to_csv(output_path, index=False)
print(f"Columns removed: {columns_to_remove}")
print(f"Cleaned CSV file saved to: {output_path}")

Columns removed: ['Margin of Error!!Total!!Population for whom poverty status is determined', 'Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years', 'Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years', 'Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years', 'Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years', 'Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years', 'Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years', 'Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years', 'Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years', 'E

C:\Users\rra3\AppData\Local\Temp\ipykernel_34756\4107992382.py:3: DtypeWarning: Columns (248,249,250,251,252,253,254,255,296,297,340,341,352,353,354,355,356,357,364,365) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [49]:
merged_df.head(20)

,8-digit CUSIP,9-digit CUSIP,STNAME,CTYNAME,Joint action,Name of Issuer,Gross Spread (per thousand) (USD),Reported Gross Spread (per thousand) (USD),SDC Estimated Gross Spread (per thousand) (USD),Management fee (USD),...,PropBlackTA,AntiAffirm,Racial Resent,Google Search,Change in Vote,Tweets,sum,number of bond issues,number by hbcu,tax privilege
0,166087LR,166087LR5,Pennsylvania,Delaware,False,54525,NaN,NaN,NaN,NaN,...,0.230410,19.0,23.0,3.0,32.0,23.5,100.5,334.0,1.0,2.97
1,nan,NaN,Texas,Rockwell,False,225650,NaN,NaN,NaN,NaN,...,NaN,5.0,8.0,27.5,26.0,16.0,82.5,121.0,6.0,0.00
2,93978PNZ,93978PNZ6,Washington,King,False,335825,NaN,8.50,NaN,NaN,...,0.070937,44.0,45.0,40.0,29.0,34.0,192.0,72.0,0.0,0.00
3,nan,NaN,New York,Sullivan,False,462200,NaN,NaN,NaN,NaN,...,0.107966,41.0,41.0,12.5,41.5,23.5,159.5,283.0,0.0,7.72
4,639438K9;639438L2;639438L3;639438L4;639438L5;6...,639438K90;639438L24;639438L32;639438L40;639438...,Pennsylvania,Northampton,False,215050,NaN,7.50,NaN,NaN,...,0.074621,19.0,23.0,3.0,32.0,23.5,100.5,334.0,1.0,2.97
5,155213LB,155213LB5,New York,Oswego,False,370575,NaN,NaN,NaN,NaN,...,0.012150,41.0,41.0,12.5,41.5,23.5,159.5,283.0,0.0,7.72
6,nan,NaN,Massachusetts,Worcester,False,289115,NaN,NaN,NaN,NaN,...,0.063256,48.0,49.0,37.0,36.5,29.5,200.0,223.0,0.0,5.47
7,899630DV,899630DV6,Oklahoma,Tulsa,False,563251,NaN,NaN,NaN,NaN,...,0.107586,9.0,15.0,22.0,36.5,11.5,94.0,60.0,1.0,0.00
8,932610RR;932610RS;932610RT;932610RU;932610RV;9...,932610RR1;932610RS9;932610RT7;932610RU4;932610...,New York,Ulster,False,449925,NaN,NaN,NaN,NaN,...,0.072986,41.0,41.0,12.5,41.5,23.5,159.5,283.0,0.0,7.72
9,932610SM,932610SM1,New York,Ulster,False,449925,NaN,NaN,NaN,NaN,...,0.072986,41.0,41.0,12.5,41.5,23.5,159.5,283.0,0.0,7.72


In [50]:
poveducmerge = pd.merge(povdf_cleaned,educdf_cleaned, on=['CTYNAME','STNAME'], how='left')
poverty_merge = pd.merge(merged_df, povdf_cleaned, on=['CTYNAME','STNAME'], how='left')
educ_merge = pd.merge(poverty_merge, educdf_cleaned, on=['CTYNAME','STNAME'], how='left')



In [51]:
educ_merge.to_csv("fuckmeplease.csv")
poveducmerge.to_csv("mergepoveduc.csv")

In [52]:
percapinc = pd.read_csv(r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\demog\percapinccounties.csv")


In [61]:
merged_df.head(20)

columns_to_remove = [col for col in educ_merge.columns if 'Geography' in col or 'RACE' in col or 'Female' in col or 'Male' in col or 'MEDIAN' in col or 'years' in col or "UNRELATED" in col]

educ_merge = educ_merge.drop(columns=columns_to_remove)
exportdf = educ_merge.copy()
finalexport = pd.merge(exportdf,percapinc, on=['CTYNAME','STNAME'], how='left')


# List of columns to be converted to float
columns_to_convert = [
    '18-24 Less than High School',
    '18-24 High School Graduate',
    '18-24 Some College or Associate\'s',
    '18-24 Bachelor\'s or Higher',
    '25+ Less than 9th Grade',
    '25+ 9th to 12th No Diploma',
    '25+ High School Graduate',
    '25+ Some College No Degree',
    '25+ Associate\'s Degree',
    '25+ Bachelor\'s Degree',
    '25+ Graduate or Professional Degree',
    '25+ High School or Higher',
    '25+ Bachelor\'s or Higher',
    '25-34 High School or Higher',
    '25-34 Bachelor\'s or Higher',
    '35-44 High School or Higher',
    '35-44 Bachelor\'s or Higher',
    '45-64 High School or Higher',
    '45-64 Bachelor\'s or Higher',
    '65+ High School or Higher',
    '65+ Bachelor\'s or Higher'
]

# Convert the specified columns to float
for column in columns_to_convert:
    finalexport[column] = pd.to_numeric(finalexport[column], errors='coerce')

for column in columns_to_convert:
    exportdf[column] = pd.to_numeric(exportdf[column], errors='coerce')
bool_columns = finalexport.select_dtypes(include='bool').columns
finalexport[bool_columns] = finalexport[bool_columns].astype(int)
# Check the data types to confirm the conversion
print(finalexport.dtypes)
finalexport = finalexport.drop(columns=['clone'])

finalexport.to_stata(r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\stata\countymuni.dta",version=117)
merged_df.to_stata(r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\stata\MuniDemo.dta",version=117)

8-digit CUSIP       object
9-digit CUSIP       object
STNAME              object
CTYNAME             object
Joint action         int32
                    ...   
staterankdollar    float64
pchange2021        float64
pchange2022        float64
rankpchange2022    float64
clone               object
Length: 125, dtype: object


C:\Users\rra3\AppData\Local\Temp\ipykernel_34756\3846005288.py:47: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    8-digit CUSIP   ->   _8_digit_CUSIP
    9-digit CUSIP   ->   _9_digit_CUSIP
    Joint action   ->   Joint_action
    Name of Issuer   ->   Name_of_Issuer
    Gross Spread (per thousand) (USD)   ->   Gross_Spread__per_thousand___USD
    Reported Gross Spread (per thousand) (USD)   ->   Reported_Gross_Spread__per_thous
    SDC Estimated Gross Spread (per thousand) (USD)   ->   SDC_Estimated_Gross_Spread__per_
    Management fee (USD)   ->   Management_fee__USD_
    Underwriters' fee (USD)   ->   Underwriters__fee__USD_
    Bid Type   ->   Bid_Type
    Sinking fund flag   ->   Sinking_fund_flag
    Callable Issue   ->   Callable_Issue
    All Managers Code   ->   All_Managers_Code
    All Use of Proceeds   ->   All_Use_of_Proceeds
    All Use of Proceeds - Bond Buyer   ->   All_Use_of_Proceeds___